<a href="https://colab.research.google.com/github/ErykCud/MLinFin2024/blob/main/CASE_ANOMALIES_CODER_ENCODER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import regularizers, optimizers
from keras import losses
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Dropout, Embedding, LSTM
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing import sequence

from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils import to_categorical

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import tensorflow
import sys

In [2]:
path = '/content/drive/MyDrive/Kopia creditcard.csv'
df = pd.read_csv(path, sep=",", index_col=None)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# Standardize
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1, 1))

In [4]:
anomalies = df[df["Class"] == 1]
normal = df[df["Class"] == 0]

anomalies.shape, normal.shape

((492, 31), (284315, 31))

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from keras.optimizers import Adam

for f in range(0, 20):
    normal = normal.sample(frac=1).reset_index(drop=True)

train_normal = normal[:2000]

data_set = pd.concat([normal[2000:], anomalies])

x_train, x_test = train_test_split(data_set, test_size=0.4, random_state=42)

x_train = x_train.sort_values(by=['Time'])
x_test = x_test.sort_values(by=['Time'])

y_train = x_train.pop('Class')
y_test = x_test.pop('Class')

x_train = x_train.drop(columns=['Time'])
x_test = x_test.drop(columns=['Time'])

scaler = StandardScaler()
x_train = scaler.fit_transform(train_normal.drop(columns=['Time', 'Class']))
x_test = scaler.transform(x_test)

input_dim = x_train.shape[1]


encoding_dim = 18
hidden_dim1 = 10
hidden_dim2 = 6
learning_rate = 1e-7
nb_epoch = 50
batch_size = 128


input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim1, activation="relu")(encoder)
encoder = Dense(hidden_dim2, activation="relu")(encoder)
decoder = Dense(hidden_dim1, activation="relu")(encoder)
decoder = Dense(encoding_dim, activation="relu")(decoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer=Adam(lr=learning_rate), loss="mean_squared_error", metrics=["mean_squared_error"])

autoencoder.fit(
    x_train, x_train,
    epochs=nb_epoch,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(x_test, x_test)
)


mse = autoencoder.evaluate(x_test, x_test)
print(f"Test MSE: {mse}")



Epoch 1/50
12/12 [==============================] - 3s 48ms/step - loss: 1.2554 - mean_squared_error: 1.2554 - val_loss: 1.0963 - val_mean_squared_error: 1.0963
Epoch 2/50
12/12 [==============================] - 0s 17ms/step - loss: 1.2449 - mean_squared_error: 1.2449 - val_loss: 1.0894 - val_mean_squared_error: 1.0894
Epoch 3/50
12/12 [==============================] - 0s 16ms/step - loss: 1.2378 - mean_squared_error: 1.2378 - val_loss: 1.0816 - val_mean_squared_error: 1.0816
Epoch 4/50
12/12 [==============================] - 0s 26ms/step - loss: 1.2279 - mean_squared_error: 1.2279 - val_loss: 1.0682 - val_mean_squared_error: 1.0682
Epoch 5/50
12/12 [==============================] - 0s 12ms/step - loss: 1.2090 - mean_squared_error: 1.2090 - val_loss: 1.0409 - val_mean_squared_error: 1.0409
Epoch 6/50
12/12 [==============================] - 0s 12ms/step - loss: 1.1703 - mean_squared_error: 1.1703 - val_loss: 0.9932 - val_mean_squared_error: 0.9932
Epoch 7/50
12/12 [================